# Imports

In [1]:
import pandas as pd, numpy as np
from crypto_trading.db import DatabaseHandler
from duckdb import sql

# Get Data

## Connect to Database

In [2]:
db = DatabaseHandler()

## Get OHLC Data

In [3]:
# Volume=Weighted Data by Instrument
query_ohlc_1d = """
    SELECT 
        instrument,
        datetime,
        -- high - low AS range,
        -- range / open AS range_perc,
        SUM(open * volume) / SUM(volume) AS open,
        SUM(high * volume) / SUM(volume) AS high, 
        SUM(low * volume) / SUM(volume) AS low,
        SUM(close * volume) / SUM(volume) AS close,
        SUM(volume) as volume,
        -- SUM(range * volume) / SUM(range) AS range,
        -- SUM(range_perc * volume) / SUM(range_perc) AS range_perc,

    FROM amberdata.ohlcv_perps_1d
    WHERE 
        CAST(datetime AS TIMESTAMP) >= NOW() - INTERVAL 101 DAY
        AND instrument IN (
            SELECT instrument
            FROM amberdata.exchange_reference
            WHERE
                exchange_enabled = TRUE
                AND contract_period = 'perpetual'
                AND quote_symbol = 'USDT'
            )
    GROUP BY instrument, datetime
    ORDER BY instrument ASC, datetime DESC
"""
df_ohlc_1d = db.query_to_df(query_ohlc_1d)

In [20]:
query_highest_volume = """
WITH LatestDatePerInstrument AS (
    SELECT
        ohlcv.instrument,
        MAX(ohlcv.datetime) AS latest_datetime
    FROM
        amberdata.ohlcv_perps_1d ohlcv
    JOIN
        amberdata.ohlcv_info_futures info
    ON
        ohlcv.exchange = info.exchange
        AND ohlcv.instrument = info.instrument
    JOIN
        amberdata.exchange_reference ref
    ON
        ohlcv.exchange = ref.exchange
        AND ohlcv.instrument = ref.instrument
    WHERE
        info.active = true
        AND ref.exchange_enabled = true
        AND ref.quote_symbol = 'USDT'
    GROUP BY
        ohlcv.instrument
),
VolumeRanked AS (
    SELECT
        ohlcv.instrument,
        ohlcv.exchange,
        ohlcv.volume,
        ROW_NUMBER() OVER (PARTITION BY ohlcv.instrument ORDER BY ohlcv.volume DESC) as volume_rank
    FROM
        amberdata.ohlcv_perps_1d ohlcv
    JOIN
        LatestDatePerInstrument ldi
    ON
        ohlcv.instrument = ldi.instrument
        AND ohlcv.datetime = ldi.latest_datetime
    JOIN
        amberdata.ohlcv_info_futures info
    ON
        ohlcv.exchange = info.exchange
        AND ohlcv.instrument = info.instrument
    JOIN
        amberdata.exchange_reference ref
    ON
        ohlcv.exchange = ref.exchange
        AND ohlcv.instrument = ref.instrument
    WHERE
        info.active = true
        AND ref.exchange_enabled = true
)
SELECT
    instrument,
    exchange,
    volume
FROM
    VolumeRanked
WHERE
    volume_rank = 1
ORDER BY
    volume DESC;
"""
df_highest_volume = db.query_to_df(query_highest_volume)

In [21]:
df_highest_volume.sort_values(['instrument', 'exchange']).iloc[50:100]

,instrument,exchange,volume
78,ALCHUSDT,binance,2.187721e+09
386,ALEOUSDT,bybit,2.859424e+07
111,ALGOUSDT,binance,1.023059e+09
345,ALICEUSDT,binance,5.122528e+07
303,ALPACAUSDT,binance,8.972117e+07
188,ALPHAUSDT,binance,3.260948e+08
124,ALTUSDT,binance,8.410902e+08
355,ALUUSDT,bybit,4.709072e+07
55,AMBUSDT,binance,3.325283e+09
46,ANIMEUSDT,binance,4.578285e+09


In [24]:
df_highest_volume.loc[df_highest_volume['instrument'].str[0]=='1']

,instrument,exchange,volume
0,1000SATSUSDT,binance,2.301905e+12
3,1000WHYUSDT,binance,2.693455e+11
4,1000PEPEUSDT,binance,2.638636e+11
8,1MBABYDOGEUSDT,binance,6.398400e+10
9,1000CHEEMSUSDT,binance,5.903814e+10
10,1000SHIBUSDT,binance,4.646193e+10
16,1000BONKUSDT,binance,2.082911e+10
18,10000SATSUSDT,bybit,1.927737e+10
24,10000LADYSUSDT,bybit,1.473488e+10
26,1000000BABYDOGEUSDT,bybit,1.314434e+10


In [31]:
sql("select distinct instrument, exchange from df_ohlc_1d where instrument like '%ENA%'")

BinderException: Binder Error: Referenced column "exchange" not found in FROM clause!
Candidate bindings: "df_ohlc_1d.open", "df_ohlc_1d.close"